In [69]:
#!/usr/bin/env python3
from ib_insync import *
from time import sleep
from datetime import datetime, time, timedelta
import pytz
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

Gateway=0
Live =0
symbol = 'GC'

if Gateway==0:
    port = 7497
    if Live==1:
        port = 7496
elif Gateway==1:
    port = 4002
    if Live==1:
        port = 4001
clientId = 2

util.startLoop()
ib = IB()
ib.connect("127.0.0.1", 4002, clientId)

<IB connected to 127.0.0.1:4002 clientId=2>

In [64]:
allData = pd.DataFrame(columns=['time', 'bid_p', 'ask_p', 'bid_v', 'ask_v'])
allData.set_index('time', inplace = True)
allData.to_hdf('/home/cl/google drive/gc_combined_tick.h5', key = 'df', complevel = 9, complib = 'zlib')

In [116]:
#only need data from 13:00 to 17:00 London time daily. Save each month, append a hdf5 with the year-month as key.
#contract = ContFuture('GC', exchange='NYMEX')
#est=pytz.timezone('America/New_York')
#allTicks = []
def pull_date(dateFrom, est, contract):
    dateFrom = dateFrom + pd.Timedelta('7 hours')
    dateTo = dateFrom + pd.Timedelta('6 hours')
    origdateTo = dateTo.tz_localize(est).strftime("%H:%M:%S")
    origdateFrom = dateFrom.tz_localize(est).strftime("%H:%M:%S")
    dateFrom = dateFrom.tz_localize(est).strftime("%Y%m%d %H:%M:%S")
    dateTo = dateTo.tz_localize(est).strftime("%Y%m%d %H:%M:%S")
    print("grabbing data from {} to {}".format(dateFrom, dateTo))
    allTicks = []
    while dateTo>=dateFrom:
        ticks = ib.reqHistoricalTicks(contract, '', dateTo, 1000,
                                      'BID_ASK', useRth=True,ignoreSize=False) #use True for less data
        allTicks = ticks[1:] + allTicks
        dateTo = allTicks[0].time.astimezone(est).strftime("%Y%m%d %H:%M:%S")
        print(dateTo)
    
    allData = pd.DataFrame([[l.time.astimezone(est).replace(tzinfo=None), l.priceBid, l.priceAsk, l.sizeBid,l.sizeAsk] 
                            for l in allTicks],
                           columns=['time','bid_p','ask_p','bid_v','ask_v'])
    allData.set_index("time", inplace=True) #replace index with datetime
    return(allData.between_time(origdateFrom, origdateTo))
    

In [ ]:
datestopull = pd.bdate_range(start = '2018-01-01', end='2018-11-01')
contract = ContFuture('GC', exchange='NYMEX')
est=pytz.timezone('America/New_York')

savelocData = "/home/cl/google drive/goldbug/"
interval = 'TICK_ALL'
symbol = 'GC_contfut'

big_data = []
write_month = str(datestopull[0].year) + '-' + str(datestopull[0].month)
for date_index in range(0, len(datestopull)):
    currentmonth = str(datestopull[date_index].year) + '-' + str(datestopull[date_index].month)
    if (currentmonth != write_month):
        print('saving data')
        write_data = pd.concat(big_data)
        write_data.to_hdf(savelocData+symbol+'_'+interval+'_'+write_month,
               key='df', complevel=9, complib='zlib')
        big_data = []
        write_month = currentmonth
    big_data = big_data + [pull_date(datestopull[date_index], est, contract)]
    
write_data = pd.concat(big_data)
write_data.to_hdf(savelocData+symbol+'_'+interval+'_'+ write_month,
                key='df', complevel=9, complib='zlib')
big_data = []

grabbing data from 20180101 07:00:00 to 20180101 13:00:00
20171229 16:45:32
grabbing data from 20180102 07:00:00 to 20180102 13:00:00
20180102 12:56:48
20180102 12:53:32
20180102 12:50:00
20180102 12:47:17
20180102 12:42:38
20180102 12:40:09
20180102 12:36:30
20180102 12:33:11
20180102 12:29:51
20180102 12:27:57
20180102 12:25:09
20180102 12:22:01
20180102 12:19:18
20180102 12:15:37
20180102 12:12:04
20180102 12:07:35
20180102 12:05:10
20180102 12:02:33
20180102 12:00:05
20180102 11:58:41
20180102 11:56:10
20180102 11:54:01
20180102 11:52:02
20180102 11:49:48
20180102 11:47:50
20180102 11:44:49
20180102 11:42:04
20180102 11:38:20
20180102 11:35:31
20180102 11:33:51
20180102 11:30:58
20180102 11:28:18
20180102 11:25:55
20180102 11:23:45
20180102 11:22:08
20180102 11:20:46
20180102 11:19:12
20180102 11:17:00
20180102 11:15:21
20180102 11:13:33
20180102 11:12:01
20180102 11:10:00
20180102 11:07:13
20180102 11:04:07
20180102 11:00:49
20180102 10:57:54
20180102 10:55:25
20180102 10:53:14
20